# Machine Learning - Decision Trees

> Heuristics for learning decision trees and their theoretical properties. 

- toc: true
- badges: true
- comments: false
- categories: ['Machine Learning','Decision Trees','Random Forests']
- image: images/decision-tree-example.jpg

# Introduction

## Classification vs. Regression

We start our discussion of decision trees with a definition of *classification* and *classifier*.

> Definition: **Classification** is the process of grouping data into discrete categories (i.e. **class labels**).

We may contrast this definition with *regression* which is the process of predicting a *continous* (i.e. real or complex-valued) output. 

A common example of a classification problem is the sorting of emails into the binary categories of *'spam'* and *'not spam'*. However, the labels in a classification problem need not be binary — they may be any discrete set. Whereas a common example of regression is learning a linear (or a non-linear) function that best fits a given dataset. 

> Note: The line between classification and regression is sometimes blurred. For instance, *logistic regression* is a regression algorithm which outputs a prediction in the continous probability range $[0,1]$. It's commonly used with a *decision rule* which casts its output into discrete classes. Thus, even though it's a regression algorithm, it can easily be converted into a classification algorithm.



> Definition: A **classifier** is any algorithm that performs classification.


## Decision Trees

*Decision trees* are one type of classifier among many. In the following discussion, for simplicity, we assume binary input and binary output for decision trees. That is, the training set is  ${S = \{(x^1,y^1), ... ,(x^k, y^k)\}$ with $x^i \in \{0,1\}^n$ and $y^i \in \{0,1\} \ \ \forall i}$ and the decision tree itself is simply a binary function. The nodes of a decision tree correspond to the *features* of the input and its leaves corrspond to the class labels. Its paths corrspond to the *conjunction of features* that lead to the class labels at the leaves.

To understand this, let's look at an example of a non-binary decision tree that's nonetheles very simple to understand because of the historical context of the data. 
